## Data processing

This Jupyter Noterbook helps us to convert binary attribute(s) to +/-1, categorical attributes(s) to onehot.

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

We load the data which were cleaned from the `data cleaning` step.

In [2]:
Xy = np.loadtxt('cryo_cleaned.dat', dtype = 'str')

print(Xy.shape)
print(Xy)

(90, 7)
[['1.0' '35.0' '12.0' '5.0' '1.0' '100.0' '0.0']
 ['1.0' '29.0' '7.0' '5.0' '1.0' '96.0' '1.0']
 ['1.0' '50.0' '8.0' '1.0' '3.0' '132.0' '0.0']
 ['1.0' '32.0' '11.75' '7.0' '3.0' '750.0' '0.0']
 ['1.0' '67.0' '9.25' '1.0' '1.0' '42.0' '0.0']
 ['1.0' '41.0' '8.0' '2.0' '2.0' '20.0' '1.0']
 ['1.0' '36.0' '11.0' '2.0' '1.0' '8.0' '0.0']
 ['1.0' '59.0' '3.5' '3.0' '3.0' '20.0' '0.0']
 ['1.0' '20.0' '4.5' '12.0' '1.0' '6.0' '1.0']
 ['2.0' '34.0' '11.25' '3.0' '3.0' '150.0' '0.0']
 ['2.0' '21.0' '10.75' '5.0' '1.0' '35.0' '0.0']
 ['2.0' '15.0' '6.0' '2.0' '1.0' '30.0' '1.0']
 ['2.0' '15.0' '2.0' '3.0' '1.0' '4.0' '1.0']
 ['2.0' '15.0' '3.75' '2.0' '3.0' '70.0' '1.0']
 ['2.0' '17.0' '11.0' '2.0' '1.0' '10.0' '0.0']
 ['2.0' '17.0' '5.25' '3.0' '1.0' '63.0' '1.0']
 ['2.0' '23.0' '11.75' '12.0' '3.0' '72.0' '0.0']
 ['2.0' '27.0' '8.75' '2.0' '1.0' '6.0' '0.0']
 ['2.0' '15.0' '4.25' '1.0' '1.0' '6.0' '1.0']
 ['2.0' '18.0' '5.75' '1.0' '1.0' '80.0' '1.0']
 ['1.0' '22.0' '5.5' '2.0' '1.0' '

### Attributes

We find number of unique value for each column, to have an idea about which variables are continuous, which variables are binary, category. It depends on data, however it is likely that nu = 2 --> binary; nu = 3 or 4: --> category, n > 4: continuous. Of course, we have to see data in detail as well.

In [3]:
X = Xy[:,:-1]
l,n = X.shape
nu = np.array([len(np.unique(X[:,i])) for i in range(n)])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[ 2 24 36 12  3 22]


In [4]:
i_binary = []
i_category = []
i_continuous = []
for i in range(X.shape[1]):
    nu = np.unique(X[:,i])
    if len(nu) == 2: # binary 
        i_binary.append(i)
    elif len(nu) < 5:
        i_category.append(i)
    else:
        i_continuous.append(i)
        
print('i_binary:',i_binary)
print('i_category:',i_category)

i_binary: [0]
i_category: [4]


We then define variable type, 1: continuous, 2: binary, 3: category.

In [5]:
variable_type  = np.ones(n) # continuous
variable_type[i_binary] = 2 # binary
variable_type[i_category] = 3 # categorical

print(variable_type)

[2. 1. 1. 1. 3. 1.]


We now convert binary to +/-1, category to onehot.

In [6]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """
    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # continuous
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))
        elif i_type == 2: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))
        else: # category      
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))        

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [7]:
# convert X
X_new = convert_binary_and_category(X,variable_type)

print(X_new.shape)
print(X_new)

(90, 8)
[[-1.000e+00  3.500e+01  1.200e+01  5.000e+00  1.000e+00  0.000e+00
   0.000e+00  1.000e+02]
 [-1.000e+00  2.900e+01  7.000e+00  5.000e+00  1.000e+00  0.000e+00
   0.000e+00  9.600e+01]
 [-1.000e+00  5.000e+01  8.000e+00  1.000e+00  0.000e+00  0.000e+00
   1.000e+00  1.320e+02]
 [-1.000e+00  3.200e+01  1.175e+01  7.000e+00  0.000e+00  0.000e+00
   1.000e+00  7.500e+02]
 [-1.000e+00  6.700e+01  9.250e+00  1.000e+00  1.000e+00  0.000e+00
   0.000e+00  4.200e+01]
 [-1.000e+00  4.100e+01  8.000e+00  2.000e+00  0.000e+00  1.000e+00
   0.000e+00  2.000e+01]
 [-1.000e+00  3.600e+01  1.100e+01  2.000e+00  1.000e+00  0.000e+00
   0.000e+00  8.000e+00]
 [-1.000e+00  5.900e+01  3.500e+00  3.000e+00  0.000e+00  0.000e+00
   1.000e+00  2.000e+01]
 [-1.000e+00  2.000e+01  4.500e+00  1.200e+01  1.000e+00  0.000e+00
   0.000e+00  6.000e+00]
 [ 1.000e+00  3.400e+01  1.125e+01  3.000e+00  0.000e+00  0.000e+00
   1.000e+00  1.500e+02]
 [ 1.000e+00  2.100e+01  1.075e+01  5.000e+00  1.000e+00  0.00

### Target

In [8]:
## target
y = Xy[:,-1].astype(float)

print(np.unique(y,return_counts=True))

(array([0., 1.]), array([42, 48]))


In [9]:
y_new = y
# convert taget to 0 and 1
#y_new = np.ones(y.shape[0])


print(np.unique(y_new,return_counts=True))

(array([0., 1.]), array([42, 48]))


In [10]:
# combine X and y
Xy_new = np.hstack((X_new,y_new[:,np.newaxis]))

In [11]:
np.savetxt('cryo_processed.dat',Xy_new,fmt='%f')